In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
import skimage.io as io
from PIL import Image
from PIL.ExifTags import TAGS
import re

dir_data = Path('data')
dir_savemeta = Path('meta')

for dir_ in [dir_data, dir_savemeta]:
    dir_.mkdir(exist_ok=True, parents=True)

print(pd.__version__)

1.0.5


## 一、比對MC metadata與蝶蛾網物種名錄

### 1、讀取各來源資料做基本整理
- MC的metadata
    - ['MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv']('https://raw.githubusercontent.com/YunghuiHsu/Moth_Project/main/Moth_thermal/data/MC-SJ-CA_ThermalTolerance-Range%20Size%20-%20MCTT%26RS.csv')
- 蝶蛾網資料 :
    - image_path.csv、photo.csv、species.csv
- 將上述三檔案合併為一份  
    - 使用'path'作為key，連結image_path
    - 使用photos的'value'與species的namecode做關聯

In [39]:
# file from local host
# file_MCTT = Path('../MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv')
# df_MC = pd.read_csv(file_MCTT)

# file from google cloud
# id for "MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS" in google cloud
id='15g7-jwKbZtGZzIOcUyiIsjFdTgStqZ8f'  
url= f"https://docs.google.com/uc?id={id}&export=download"
df_MC = pd.read_csv(url)
df_MC

df_MCTT = df_MC[df_MC.Event == 'MCTT']
df_MCRS = df_MC[df_MC.Event == 'MCRS']

species_list = df_MC[df_MC.Species.notnull()].Species.sort_values().unique()
print('Total species in "MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv" : ', len(species_list))
print('Number of species in MCTT : ', df_MCTT.Species.nunique())
print('Number of species in MCRS : ', df_MCRS.Species.nunique())

species_list = sorted([s for s in species_list])
species_list[:10]

Total species in "MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv" :  964
Number of species in MCTT :  366
Number of species in MCRS :  841


['"Adrapsa" sp.',
 '"Collix" sp.',
 '"Eilema" sp. 1',
 '"Eilema" sp. 2',
 '"Eilema" sp. 3',
 '"Koyaga" sp._blue',
 '"Lophophleps" sp.',
 '"Plecoptera" sp.',
 '"Rivula" sp.',
 '"Syncosmia" sp.']

##### df_path

In [71]:
df_path = pd.read_csv('data/image_path.csv', sep='\t', index_col=0)
df_path

,path
path_id,
1,./images/Noctuoidea 夜蛾總科/ERE/Calliteara saiton...
2,./images/Noctuoidea 夜蛾總科/ERE/Condate angulina_...
3,./images/Noctuoidea 夜蛾總科/ERE/ERE_EUB/150714 基隆...
4,./images/Noctuoidea 夜蛾總科/ERE/ERE_EUB/101203 宜蘭...
5,./images/Noctuoidea 夜蛾總科/ERE/ERE_EUB/111227 嘉義...
...,...
19348,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/160402 貢寮...
19349,./images/Noctuoidea 夜蛾總科/ERE/ERE_ERE/Polydesma...
19350,./images/Noctuoidea 夜蛾總科/NOC/NOC_BAG/121104 台北...


##### df_photos

In [72]:
df_photos = pd.read_csv('data/photos.csv', sep='\t', index_col=0)
print(len(df_photos))
assert (df_photos.value == df_photos.value_txt).all()
print(df_photos.field.unique())
print(df_photos.value.unique())
print(df_photos.value.nunique())
df_photos[:3]

12301
[216]
['345818' 'sci_hash_cda7310418f257efcca11a381b554f7b' '346272' ...
 '434531' '347343' '347336']
3057


,field,value,value_txt,modified
path,,,,
1,216,345818,345818,2021-01-26 10:01:03
2,216,sci_hash_cda7310418f257efcca11a381b554f7b,sci_hash_cda7310418f257efcca11a381b554f7b,2021-01-26 10:01:06
3,216,346272,346272,2021-01-26 10:01:06


In [73]:
df_photos_ = df_photos['value']
df_photos_.name = 'namecode'
df_photos_ = df_photos_.reset_index()
df_photos_.set_index('namecode', inplace=True)
df_photos_.rename(columns={'path' : 'path_id'}, inplace=True)
df_photos_

,path_id
namecode,
345818,1
sci_hash_cda7310418f257efcca11a381b554f7b,2
346272,3
346273,4
346273,5
...,...
347335,19326
347322,19329
347341,19330


##### df_species

In [74]:
df_species = pd.read_csv('data/species.csv', sep='\t', index_col=0)
assert (df_species.value == df_species.value_txt).all()
print(len(df_species))

print(df_species.field.unique())
print(df_species.value.unique())
print('Total species :', df_species.value.nunique())
df_species

8613
[26]
['Abraximorpha davidii ermasis' 'Aeromachus bandaishanus'
 'Aeromachus inachus formosana' ... 'Phazaca theclata'
 'Chilades lajus koshunensis' 'Shijimia moorei taiwana']
Total species : 8499


,field,value,value_txt,modified
namecode,,,,
345448,26,Abraximorpha davidii ermasis,Abraximorpha davidii ermasis,2021-01-26 11:22:31
345449,26,Aeromachus bandaishanus,Aeromachus bandaishanus,2021-01-26 11:21:26
345450,26,Aeromachus inachus formosana,Aeromachus inachus formosana,2021-01-26 11:20:47
345451,26,Aeromachus matudai,Aeromachus matudai,2021-01-26 09:59:16
345452,26,Ampittia dioscorides etura,Ampittia dioscorides etura,2021-01-26 11:22:09
...,...,...,...,...
sci_hash_d54dbf29060813fcc0f36b81227d32c3,26,Hypephyra terrosa,Hypephyra terrosa,2021-01-26 10:52:50
sci_hash_7914275b0cc47a4935aaf497ce2f4796,26,Mimochroa olivescens,Mimochroa olivescens,2021-01-26 10:58:17
sci_hash_249499f05ba624c4da0d4544bcb0a8ef,26,Phazaca theclata,Phazaca theclata,2021-01-26 11:03:14


In [76]:
df_species_ = df_species['value']
df_species_.name = 'species'
df_species_


namecode
345448                                       Abraximorpha davidii ermasis
345449                                            Aeromachus bandaishanus
345450                                       Aeromachus inachus formosana
345451                                                 Aeromachus matudai
345452                                         Ampittia dioscorides etura
                                                         ...             
sci_hash_d54dbf29060813fcc0f36b81227d32c3               Hypephyra terrosa
sci_hash_7914275b0cc47a4935aaf497ce2f4796            Mimochroa olivescens
sci_hash_249499f05ba624c4da0d4544bcb0a8ef                Phazaca theclata
sci_hash_e3f6479f537d7d41c16916469576f104      Chilades lajus koshunensis
sci_hash_4f70786a719f7c30133212098f431722         Shijimia moorei taiwana
Name: species, Length: 8613, dtype: object

In [77]:
df_ = pd.merge(df_species_, df_photos_,
               left_index=True, right_index=True,
               how='left')
print(df_[df_.species.isnull()].size)
print('namecode with image_path :', df_[df_.path_id.notnull()].path_id.size)
print('Total species :', df_.species.nunique())
df_ = df_.reset_index()
df_


0
namecode with image_path : 12301
Total species : 8499


,namecode,species,path_id
0,28266ea08346b4dea16153ab050dbbfc,Rhesala inconcinnalis,NaN
1,29d6e207436123895a445410f0ec4c0f,Micreremites japonibia,NaN
2,2c0ed1c0c3463bdb1a6dbd490b9d128d,Paracrama latimargo,NaN
3,2e78ae7737ad41a8f8e3148c0e926d8b,Gesonia fallax,NaN
4,343743,Acrolepiopsis brevipenella,NaN
...,...,...,...
17852,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1574.0
17853,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1598.0
17854,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1624.0
17855,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1827.0


In [78]:

df_dearlep = df_.merge(df_path, left_on='path_id', right_index=True, how='outer').reset_index(drop=True)
df_dearlep

,namecode,species,path_id,path
0,28266ea08346b4dea16153ab050dbbfc,Rhesala inconcinnalis,NaN,NaN
1,29d6e207436123895a445410f0ec4c0f,Micreremites japonibia,NaN,NaN
2,2c0ed1c0c3463bdb1a6dbd490b9d128d,Paracrama latimargo,NaN,NaN
3,2e78ae7737ad41a8f8e3148c0e926d8b,Gesonia fallax,NaN,NaN
4,343743,Acrolepiopsis brevipenella,NaN,NaN
...,...,...,...,...
24866,NaN,NaN,19348.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/160402 貢寮...
24867,NaN,NaN,19349.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_ERE/Polydesma...
24868,NaN,NaN,19350.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_BAG/121104 台北...
24869,NaN,NaN,19351.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_EUB/111227 嘉義...


In [79]:
print('Total data in dearlep:', len(df_dearlep)) 
print('Total species in dearlep:', df_dearlep.species.nunique())
print('Total species in "MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv" : ', len(species_list))
print('-'*50)
print('imgs_path about dearlep : ')
print('\tTotal number of images_path(imgs) in dearlep :', len(df_path) )
print('\timages_path which species(namecode) column is null :', df_dearlep[df_dearlep.path.notnull()].species.isnull().sum())
print('\tspecies with image_path :', df_dearlep[df_dearlep.species.notnull()].path_id.notnull().values.sum())



Total data in dearlep: 24871
Total species in dearlep: 8499
Total species in "MC-SJ-CA_ThermalTolerance-Range Size  - MCTT&RS.csv" :  964
--------------------------------------------------
imgs_path about dearlep : 
	Total number of images_path(imgs) in dearlep : 19315
	images_path which species(namecode) column is null : 7014
	species with image_path : 12301


In [99]:
df_MC_ = df_MC.loc[:,['Family', 'Species','Code','Species_previous']]
df_MC_.dropna(subset=['Species'], inplace=True)
df_MC_.drop_duplicates(subset=['Family', 'Species'],inplace=True)
df_MC_

,Family,Species,Code,Species_previous
0,Noctuidae,Hermonassa legraini,346335,Hermonassa legraini
1,Noctuidae,Xestia tamsi,346826,Xestia tamsi
2,Geometridae,Alcis postlurida,344628,Alcis postlurida
3,Noctuidae,Pseudodeltote formosana,346678,Pseudodeltote formosana
5,Noctuidae,Olivenebula oberthueri,346569,Olivenebula oberthueri
...,...,...,...,...
15213,Noctuidae,Feliniopsis asahinai,346301,Feliniopsis asahinai
15219,Pyralidae,Galleria mellonella,347465,Galleria mellonella
15261,Erebidae,Hemistola simplex,344918,Hemistola simplex
15264,Erebidae,Hipoepa biasalis,346338,Hipoepa biasalis


In [86]:
df_MC_[df_MC_.duplicated(keep=False, subset='Species')].sort_values(by='Species')

,Species,Code,Species_previous
2369,"""Eilema"" sp. 3",NaN,\Eilema\ sp.
3311,"""Eilema"" sp. 3",NaN,"""Eilema"" sp."
11677,Aberrasine lichenshihi,NaN,Miltochrista lichenshihi
1949,Aberrasine lichenshihi,macro-06,Miltochrista lichenshihi
12758,Aberrasine lichenshihi,343799,Lyclene acteola
...,...,...,...
5033,Sinarella sp. 2,NaN,Sianrella huahua
5133,Stigmatophora karenkonis,432975,Stigmatophora karenkonis
1510,Stigmatophora karenkonis,macro-03,Stigmatophora karenkensis
7297,Tyana falcata,346921,Nolathripa lactaria


In [88]:
df_dearlep_dropna = df_dearlep.dropna(subset=['namecode'])
print(len(df_dearlep_dropna))
df_dearlep_dropna


17857


,namecode,species,path_id,path
0,28266ea08346b4dea16153ab050dbbfc,Rhesala inconcinnalis,NaN,NaN
1,29d6e207436123895a445410f0ec4c0f,Micreremites japonibia,NaN,NaN
2,2c0ed1c0c3463bdb1a6dbd490b9d128d,Paracrama latimargo,NaN,NaN
3,2e78ae7737ad41a8f8e3148c0e926d8b,Gesonia fallax,NaN,NaN
4,343743,Acrolepiopsis brevipenella,NaN,NaN
...,...,...,...,...
17852,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1477.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/B...
17853,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1574.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/B...
17854,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1598.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/B...
17855,sci_hash_fe4a90bf288ffe04f17f0b0c0b4ba035,Bertula adrapsoides,1624.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/B...


#### 1.1 使用namecode比對，將MC metadat與蝶蛾網資料做merge 

In [101]:
# 1. 使用namecode比對，將MC metadat與蝶蛾網資料做merge 

df_MC_imgpath_byCodeName = (df_MC_.loc[:,['Family','Species','Code']]
                            .merge(
                                df_dearlep_dropna, 
                                right_on='namecode', 
                                left_on='Code', 
                                how='left' )
                            )

df_MC_imgpath_byCodeName



,Family,Species,Code,namecode,species,path_id,path
0,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3638.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
1,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3675.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...
2,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3692.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...
3,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3757.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
4,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3781.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
...,...,...,...,...,...,...,...
3426,Erebidae,Hipoepa biasalis,346338,346338,Hipoepa biasalis,1464.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/H...
3427,Erebidae,Hipoepa biasalis,346338,346338,Hipoepa biasalis,1653.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/H...
3428,Erebidae,Hipoepa biasalis,346338,346338,Hipoepa biasalis,1727.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/H...
3429,Erebidae,Hipoepa biasalis,346338,346338,Hipoepa biasalis,1784.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/190413 台北...


In [102]:
# codename match 但 物種名不同
(df_MC_imgpath_byCodeName.query('Species != species')
.dropna(subset=['Code'])
.loc[:,['Family', 'Species','Code','species']]
)

,Family,Species,Code,species
106,Noctuidae,Nacna buschmannferenci,346539,Nacna malachitis
180,Noctuidae,Polia goliath,346662,Polia gigantea gigantea
181,Noctuidae,Polia goliath,346662,Polia gigantea gigantea
182,Noctuidae,Polia goliath,346662,Polia gigantea gigantea
183,Noctuidae,Polia goliath,346662,Polia gigantea gigantea
...,...,...,...,...
3273,Noctuidae,Iragaodes nobilis,346093,Callopistria delicata
3274,Noctuidae,Iragaodes nobilis,346093,Callopistria delicata
3275,Noctuidae,Iragaodes nobilis,346093,Callopistria delicata
3308,Crambidae,Talanga sexpunctalis,344391,Talanga nympha


#### 1.2 使用物種名比對，將MC metadat與蝶蛾網資料做merge

In [156]:
# 2. 使用物種名比對，將MC metadat與蝶蛾網資料做merge

print('Total Species in MC metadata : ', (df_MC_.Family + '_' + df_MC_.Species).nunique())
print('Total code in MC metadata : ', df_MC_.Code.nunique()) 

# 部分物種欄位僅鑑定到屬，因此code欄位是nan
print('Code column is NaN :') 
df_MC_CodeNull = df_MC_[df_MC_.Code.isnull()][['Family','Species']]
print('\tNumber of species which code is NaN :',len(df_MC_CodeNull))
# print('species column which code is NaN :', df_MC_[df_MC_.Code.isnull()].Species.unique())

# ! 部分有完整物種名的種類並沒有搭配的namecode
index_drop = df_MC_CodeNull.query("Species.str.contains('sp.')").index.values
print('\tNumber of species contains "sp.": ', len(index_drop))

df_MC_CodeNull.drop(labels=index_drop, inplace=True)
print('\tNumber of species which code is NaN but have full name : ', len(df_MC_CodeNull))
df_MC_CodeNull.reset_index(drop=True, inplace=True)
df_MC_CodeNull
# species_MC_CodeNull = df_MC_[df_MC_.Code.isnull()].Species.unique()
# species_MC_CodeNull = [sp for sp in species_MC_CodeNull if not 'sp.' in sp]
# print('\tNumber of species which code is NaN but have full name : ', len(species_MC_CodeNull))


Total Species in MC metadata :  971
Total code in MC metadata :  844
Code column is NaN :
	Number of species which code is NaN : 120
	Number of species contains "sp.":  96
	Number of species which code is NaN but have full name :  24


,Family,Species
0,Limacodidae,Narosoideus flavidorsalis
1,Erebidae,Bertula chunchunyangae
2,Zygaenidae,Morionia sciara
3,Erebidae,Sinarella subaegrota
4,Erebidae,Sinarella yuanyuan
5,Erebidae,Nodaria externalis
6,Erebidae,Sesapa kishidai
7,Geometridae,Peratophyga grata stuningi
8,Erebidae,Parasiccia limbata
9,Noctuidae,Chasminodes pseudalbonitens


In [154]:
# 從df_MC缺乏codename的物種清單，根據物種名與df_dearlep做合併
df_MC_imgpath_bySpName = df_MC_CodeNull.merge(
    df_dearlep.dropna(subset=['namecode']), 
    right_on='species', 
    left_on='Species', 
    how='left' )
df_MC_imgpath_bySpName

,Family,Species,namecode,species,path_id,path
0,Limacodidae,Narosoideus flavidorsalis,MON,Narosoideus flavidorsalis,7220.0,./images/2020/Narosoideus flavidorsalis_male_N...
1,Limacodidae,Narosoideus flavidorsalis,MON,Narosoideus flavidorsalis,7692.0,"./images/2020/Narosoideus flavidorsalis, NSMT3..."
2,Limacodidae,Narosoideus flavidorsalis,MON,Narosoideus flavidorsalis,7831.0,./images/2020/Narosoideus sp_male_Cien_7-VI-20...
3,Erebidae,Bertula chunchunyangae,sci_hash_4af39206f8f5465cf11fa856edf98491,Bertula chunchunyangae,942.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/100225 Il...
4,Erebidae,Bertula chunchunyangae,sci_hash_4af39206f8f5465cf11fa856edf98491,Bertula chunchunyangae,975.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Genita...
...,...,...,...,...,...,...
95,Erebidae,Cidariplura tsoyuanhsiehi,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1156.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Genita...
96,Erebidae,Cidariplura tsoyuanhsiehi,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1441.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...
97,Erebidae,Cidariplura tsoyuanhsiehi,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1594.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...
98,Erebidae,Cidariplura tsoyuanhsiehi,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1637.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...


In [155]:
# 檢查 df_MC_imgpath_bySpName 與 df_MC_imgpath_byCodeName 當中的資料是否有重疊
# 1. namecode	 2. Species
# df_MC_imgpath = pd.concat([df_MC_imgpath_bySpName,df_MC_imgpath_byCodeName], sort=False)

sp_bySpName = df_MC_imgpath_bySpName.Species.unique()
sp_byCodeName = df_MC_imgpath_byCodeName.Species.unique()
print('物種名重疊 : ' , len(set(sp_bySpName) & set(sp_byCodeName)))

code_bySpName = df_MC_imgpath_bySpName.namecode.unique()
code_byCodeName = df_MC_imgpath_byCodeName.namecode.unique()
print('namecode重疊 : ' , len(set(code_bySpName) & set(code_byCodeName)))

物種名重疊 :  24
namecode重疊 :  0


#### 1.3 最後資料產出
- 以namecode為key， 產出merge的資料(df_MC_imgpath_byCodeName)
- 補充MC metada中 缺乏namecode 但物種名完整的資料(df_MC_imgpath_bySpName)

In [160]:
print('df_MC_imgpath_bySpName : 缺少影像的物種:')
print(df_MC_imgpath_bySpName[
    df_MC_imgpath_bySpName.path_id.isnull()
    ].loc[:,['Family','Species','namecode','path_id']])


df_MC_imgpath_bySpName中缺少影像的物種:
      Family             Species      namecode  path_id
39  Erebidae  Parasiccia limbata  343929-s5-v2      NaN


In [162]:
print('df_MC_imgpath_byCodeName : 缺少影像的物種:')
print(df_MC_imgpath_byCodeName[df_MC_imgpath_byCodeName.path_id.isnull()]
      .loc[:,['Family','Species','namecode','path_id']]
      .path_id.size)
print(df_MC_imgpath_byCodeName[df_MC_imgpath_byCodeName.path_id.isnull()]
      .loc[:,['Family','Species','namecode','path_id']])


df_MC_imgpath_byCodeName : 缺少影像的物種:
242
           Family                          Species namecode  path_id
106     Noctuidae           Nacna buschmannferenci   346539      NaN
113     Noctuidae           Leucapamea formosensis   346427      NaN
287      Erebidae               Hypocala deflorata   346398      NaN
322      Erebidae             Asota egens confinis   346031      NaN
323    Drepanidae  Horithyatira decorata takamukui   347724      NaN
...           ...                              ...      ...      ...
3344     Erebidae                  Sinarella sp. 1      NaN      NaN
3367     Erebidae               Ammatho tairadiata   343890      NaN
3379     Erebidae        Cidariplura tsoyuanhsiehi      NaN      NaN
3406  Tortricidae                  Terthreutis sp.      NaN      NaN
3409     Erebidae           Arasada kanshireiensis   346025      NaN

[242 rows x 4 columns]


### 2、以MC metadata為基底，比對各物種影像數量
- 先以namecode與code做合併、在以物種名做合併
- 取得各物種所有影像的path後，在計算各物種有多少path(影像數)  


#### 2.1 先以namecode比對，計算各物種對應的path數量(即影像數量)
- 根據比對結果找出沒有影像的物種
- 從中挑出有完整物種名的種類

In [179]:
df_MC_SpCode = df_MC.loc[:,['Family','Species','Code']]
df_MC_SpCode = (df_MC_SpCode.sort_values(by='Species')
                .drop_duplicates(subset=['Family', 'Species'])
                .dropna(subset=['Species'])
                .reset_index(drop=True)
                )
df_MC_SpCode


,Family,Species,Code
0,Erebidae,"""Adrapsa"" sp.",NaN
1,Geometridae,"""Collix"" sp.",NaN
2,Erebidae,"""Eilema"" sp. 1",NaN
3,Erebidae,"""Eilema"" sp. 2",NaN
4,Erebidae,"""Eilema"" sp. 3",NaN
...,...,...,...
966,Noctuidae,Xylostola indistincta,346837
967,Geometridae,Yashmakia suffusa,345147
968,Erebidae,Zanclognatha nigrisigna,346843
969,Erebidae,Zanclognatha sp.,NaN


In [181]:
df_MC_imgpath = pd.concat(
    [df_MC_imgpath_byCodeName, df_MC_imgpath_bySpName],
    sort=False)
print(df_MC_imgpath.path.notnull().sum())
df_MC_imgpath.to_csv('meta/df_MC_SpCode_imgpath')
df_MC_imgpath

3288


,Family,Species,Code,namecode,species,path_id,path
0,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3638.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
1,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3675.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...
2,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3692.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...
3,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3757.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
4,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3781.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...
...,...,...,...,...,...,...,...
95,Erebidae,Cidariplura tsoyuanhsiehi,NaN,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1156.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Genita...
96,Erebidae,Cidariplura tsoyuanhsiehi,NaN,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1441.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...
97,Erebidae,Cidariplura tsoyuanhsiehi,NaN,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1594.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...
98,Erebidae,Cidariplura tsoyuanhsiehi,NaN,sci_hash_509bea05bb81430db9b58380a03f7332,Cidariplura tsoyuanhsiehi,1637.0,./images/Noctuoidea 夜蛾總科/ERE/ERE_HER/00 Spec/C...


In [185]:
path_count = df_MC_imgpath.groupby(['Family', 'Species']).path.count()
df_MC_SpCode_count = df_MC_SpCode.merge(
    path_count, left_on=['Family','Species'], right_index=True, how='left'
    ).reset_index(drop=True)
df_MC_SpCode_count.columns = ['Family','Species','Code','path_count']
print(df_MC_SpCode_count.describe())
print('Total number of paths : ', df_MC_SpCode_count.path_count.sum())
df_MC_SpCode_count

       path_count
count  971.000000
mean     3.386200
std      3.148584
min      0.000000
25%      1.000000
50%      3.000000
75%      5.000000
max     20.000000
Total number of paths :  3288


,Family,Species,Code,path_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0
1,Geometridae,"""Collix"" sp.",NaN,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0
...,...,...,...,...
966,Noctuidae,Xylostola indistincta,346837,1
967,Geometridae,Yashmakia suffusa,345147,3
968,Erebidae,Zanclognatha nigrisigna,346843,4
969,Erebidae,Zanclognatha sp.,NaN,0


In [186]:
print('971種當中，沒有影像資料的種數 : ', df_MC_SpCode_count.query('path_count == 0').__len__())
df_MC_SpCode_count.query('path_count == 0')

971種當中，沒有影像資料的種數 :  219


,Family,Species,Code,path_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0
1,Geometridae,"""Collix"" sp.",NaN,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0
...,...,...,...,...
926,Tortricidae,Terthreutis sp.,NaN,0
931,Geometridae,Timandra stueningi,345321,0
950,Crambidae,Tyspanodes striata,344406,0
953,Erebidae,Vamuna alboluteora,426799,0


In [216]:
df_sp_path_null = df_MC_SpCode_count.query('path_count ==0 and not Species.str.contains("sp\.")')
print('有完整物種名，但沒有影像的資料筆數 : ', len(df_sp_path_null))
df_sp_path_null[::15]

有完整物種名，但沒有影像的資料筆數 :  70


,Family,Species,Code,path_count
15,Geometridae,Abraxas illuminata,414096,0
244,Erebidae,Corgatha costimacula,429679,0
472,Erebidae,Hypocala violacea,346401,0
703,Erebidae,Oxacme cretacea,430036,0
866,Lasiocampidae,Somadasys catocoides,345553,0


#### 2.2 從步驟1找出的物種清單，改以物種重複上述步驟

#### 2.3 取出有完整學名，但沒有影像的物種名單後，改以物種名與蝶蛾網做merge


In [217]:
df_sp_path_null_path = df_sp_path_null.merge(
    df_dearlep, 
    left_on='Species', 
    right_on='species', 
    how='left'
)
df_sp_path_null_path

,Family,Species,Code,path_count,namecode,species,path_id,path
0,Geometridae,Abraxas illuminata,414096,0,414096,Abraxas illuminata,NaN,NaN
1,Geometridae,Acolutha pulchella semifulva,344604,0,344604,Acolutha pulchella semifulva,NaN,NaN
2,Noctuidae,Aedia acronyctoides,345953,0,345953,Aedia acronyctoides,NaN,NaN
3,Crambidae,Anania pata,349614,0,NaN,NaN,NaN,NaN
4,Erebidae,Arasada kanshireiensis,346025,0,346025,Arasada kanshireiensis,NaN,NaN
...,...,...,...,...,...,...,...,...
67,Pyralidae,Sybrida inordinata,347546,0,347546,Sybrida inordinata,NaN,NaN
68,Erebidae,Sypnoides simplex,346778,0,346778,Sypnoides simplex,NaN,NaN
69,Pyralidae,Teliphasa amica,347552,0,347552,Teliphasa amica,NaN,NaN
70,Crambidae,Tenerobotys subfumalis,344394,0,344394,Tenerobotys subfumalis,NaN,NaN


In [218]:
df_MC_imgpath2 = pd.concat(
    [df_MC_imgpath, df_sp_path_null_path]
    ,sort=False)
print(df_MC_imgpath2.path.notnull().sum())
df_MC_imgpath2.to_csv('meta/df_MC_SpCode_imgpath.csv')
df_MC_imgpath2

3290


,Family,Species,Code,namecode,species,path_id,path,path_count
0,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3638.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
1,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3675.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...,NaN
2,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3692.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...,NaN
3,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3757.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
4,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3781.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
...,...,...,...,...,...,...,...,...
67,Pyralidae,Sybrida inordinata,347546,347546,Sybrida inordinata,NaN,NaN,0.0
68,Erebidae,Sypnoides simplex,346778,346778,Sypnoides simplex,NaN,NaN,0.0
69,Pyralidae,Teliphasa amica,347552,347552,Teliphasa amica,NaN,NaN,0.0
70,Crambidae,Tenerobotys subfumalis,344394,344394,Tenerobotys subfumalis,NaN,NaN,0.0


In [236]:
path_count = df_MC_imgpath2.groupby(['Family', 'Species']).path.count()

# df_MC_SpCode_ = df_MC_SpCode.merge(df_dearlep_na.loc[:,['namecode']], left_on='Code', right_on='namecode', how='left').drop_duplicates(subset=['Species'])
df_MC_SpCode_count = df_MC_SpCode.merge(
    path_count, 
    left_on=['Family', 'Species'], 
    right_index=True, 
    how='left'
    ).reset_index(drop=True)
df_MC_SpCode_count.columns = ['Family', 'Species','Code','path_count']
print(df_MC_SpCode_count.describe())
print('Total number of paths : ', df_MC_SpCode_count.path_count.sum())
df_MC_SpCode_count.to_csv('meta/df_MC_path_count.csv')
df_MC_SpCode_count

       path_count
count  971.000000
mean     3.388260
std      3.147021
min      0.000000
25%      1.000000
50%      3.000000
75%      5.000000
max     20.000000
Total number of paths :  3290


,Family,Species,Code,path_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0
1,Geometridae,"""Collix"" sp.",NaN,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0
...,...,...,...,...
966,Noctuidae,Xylostola indistincta,346837,1
967,Geometridae,Yashmakia suffusa,345147,3
968,Erebidae,Zanclognatha nigrisigna,346843,4
969,Erebidae,Zanclognatha sp.,NaN,0


### 3、整理出缺乏影像資料的物種名、樣本編號及屬與哪個event(RS或TT)

In [221]:
print('971種當中，沒有影像資料的種數 : ', df_MC_SpCode_count.query('path_count == 0').path_count.size )
df_sp_path_null = df_MC_SpCode_count.query('path_count == 0 and ~ Species.str.contains("sp\.")')

print('有完整物種名，但沒有影像的資料筆數 : ', len(df_sp_path_null))
df_sp_path_null

971種當中，沒有影像資料的種數 :  218
有完整物種名，但沒有影像的資料筆數 :  113


,Family,Species,Code,path_count
12,Erebidae,Aberrasine lichenshihi,NaN,0
15,Geometridae,Abraxas illuminata,414096,0
25,Geometridae,Achrosis rufescens,344960,0
27,Geometridae,Acolutha pulchella semifulva,344604,0
37,Noctuidae,Aedia acronyctoides,345953,0
...,...,...,...,...
921,Pyralidae,Teliphasa amica,347552,0
925,Crambidae,Tenerobotys subfumalis,344394,0
931,Geometridae,Timandra stueningi,345321,0
950,Crambidae,Tyspanodes striata,344406,0


In [222]:
# 檢查物種名中含有"sp."的狀況
print('物種名中含有"sp."的資料量 : ', df_MC_SpCode_count.query('Species.str.contains("sp\.")').__len__())

print(df_MC_SpCode_count.query(
    'Species.str.contains("sp\.") and Code.notnull()'
    ))
print()
print(df_MC_SpCode_count.query(
    'Species.str.contains("sp\.") and path_count > 0'
    ))

物種名中含有"sp."的資料量 :  107
        Family                 Species       Code  path_count
118  Noctuidae          Atrachea sp. 1   macro-04           0
267  Crambidae             Cyana sp. 1     344123           2
325   Erebidae            Eilema sp. 2     431513           0
385   Erebidae             Ganhara sp.    Ganhara           0
499   Erebidae  Katha sp._small_yellow  Katha_s_y           0
654    Nolidae              Nola sp. 1   macro-02           0
679  Crambidae             Omiodes sp.   micro-07           0
810  Pyralidae         Pyralinae sp. 2   micro-02           0
830  Pyralidae             Salma sp. 1   micro-03           0
831  Pyralidae             Salma sp. 2   micro-04           0
832  Pyralidae             Salma sp. 3   micro-05           0

        Family      Species    Code  path_count
266   Erebidae  Cyana sp. 1     NaN           7
267  Crambidae  Cyana sp. 1  344123           2


In [225]:
index = df_MC_SpCode_count[df_MC_SpCode_count.Species.str.contains('sp\.')].index.values

df_MC_SpCode_count.iloc[index,:].query('Code.isnull() and path_count > 0')

,Family,Species,Code,path_count
266,Erebidae,Cyana sp. 1,NaN,7


## 二、比對MC_DearLep_specimen-image_20211203

#### Preprocess problemed .img data type

4220


In [2]:
# dir_MC_DearLep = Path('../data_raw/MC_DearLep_specimen-image_20220105')
# paths_MC_DearLep_ = list(dir_MC_DearLep.glob('**/*'))

dir_MCTT =  Path('../../data_raw/DearLep/MCTT')
dir_MC_DearLep_ =  Path('../../data_raw/DearLep/MC_DearLep_specimen-image_20211203')
paths_MC_DearLep_ = list(dir_MCTT.glob('**/*')) + list(dir_MC_DearLep_ .glob('**/*'))
print(len(paths_MC_DearLep_))
print(len(paths_MC_DearLep_))
suffix_set =set([path.suffix for path in paths_MC_DearLep_])
print(suffix_set)

4220
4220
{'', '.png', '.psd', '.jpg', '.jpeg'}


In [20]:

paths_MC_DearLep = [path for path in paths_MC_DearLep_ 
                    if path.suffix.lower() in ['.jpg','.jpeg','.png']]
print(len(paths_MC_DearLep))
suffix_set =set([path.suffix for path in paths_MC_DearLep])
print(suffix_set)

paths_MC_DearLep_not = [path for path in paths_MC_DearLep_ 
                    if path.suffix.lower() not in ['.jpg','.jpeg','.png']
                    ]
print(len(paths_MC_DearLep_not))
paths_MC_DearLep_not

4218
{'.png', '.jpeg', '.jpg'}
2


[PosixPath('../../data_raw/DearLep/MC_DearLep_specimen-image_20211203/Nola formosalesa_male_20171228_64.psd'),
 PosixPath('../../data_raw/DearLep/MC_DearLep_specimen-image_20211203/.DS_Store')]

In [21]:
# # 
suffix_set = set([path.suffix for path in paths_MC_DearLep_not])
print(suffix_set)

problem_path = []
for path in paths_MC_DearLep_not:
    try:
        img = Image.open(path)
    except Exception as err:
        print(err)
        problem_path.append(path)
print(len(problem_path))
# print(problem_path)

# for path in problem_path:
#     path.unlink()

{'', '.psd'}
cannot identify image file '/home/jovyan/moth_thermal_project/data/data_raw/DearLep/MC_DearLep_specimen-image_20211203/.DS_Store'
1


In [22]:
# updata .suffix problemed file
failure = []
for path in paths_MC_DearLep_not:
    fname = path.stem
    dir_ = path.parent
    if not str(fname).startswith('.') and not str(path.suffix)=='.psd' :
        print(path.name)
        
        try:
#             shutil.copyfile(path, dir_.joinpath(fname + '.jpg'))
#             path.unlink()
            print('\tDeleted and updated as : ',  fname + '.jpg')
        except Exception as e:
            print(e)
            failure.append(path)
            
print(failure)
        

[]


In [3]:
# check again
dir_MC_DearLep = Path('../../data_raw/DearLep')
paths_MC_DearLep = list(dir_MC_DearLep.glob('**/*'))
paths_MC_DearLep = [path for path in paths_MC_DearLep 
                    if path.suffix.lower() in ['.jpg', '.jpeg', '.png']]
print(len(paths_MC_DearLep))

paths_MC_DearLep = [path for path in paths_MC_DearLep 
                    if not path.stem.startswith('\.')]
print(len(paths_MC_DearLep))

suffix_set = set([path.suffix for path in paths_MC_DearLep])
print(suffix_set)

print('image number : ', len(paths_MC_DearLep))

df = pd.DataFrame(paths_MC_DearLep,columns=['path'])
df['name'] =  df.path.apply(lambda x : Path(x).stem)
del df['path']

print('number of unique filename  : ', df.name.nunique())

4218
4218
{'.jpeg', '.png', '.jpg'}
image number :  4218
number of unique filename  :  3879


#### 撈出"MC_DearLep_specimen-image_20211203"資料夾內影像的metada


- 抓出<lr:hierarchicalSubject> ... </lr:hierarchicalSubject> 區塊內的資料

In [4]:
target = r'<lr:hierarchicalSubject>'
target_end = r'</lr:hierarchicalSubject>'

err_log = {}
fname_tag = {}
e=0
for idx, path in enumerate(paths_MC_DearLep):

    
    # if idx %17 ==0:
    fname = path.stem
    print(f'id : {idx}, {fname:20s}\t\t\t\t', end='\r')
    
    # 設置errors='ignore'， 直接忽略byte資料錯誤訊息
    try:
        with open(path, mode='r', encoding='utf-8', errors='ignore') as img_file:
            lines = img_file.readlines()
            lines_ = [line.strip() for line in lines]
        
            target_ = re.split(target, str(lines_))[1]
            result = re.split(target_end, target_)[0]
            
            output_ = (result
            .replace(r'<rdf:Bag>', '').replace(r'</rdf:Bag>', '')
            .replace("'", '').replace(',', '')
            .replace(r'</rdf:li>', '').strip()
            .split(r'<rdf:li>')
            )
            # print(len(output_))
            output_ =  [str_.strip() for str_ in output_ if not str_  =='']
            output = ['Taxon|' + str_  if len(str_.split('|')) > 2 else str_ for str_ in output_ ]
            # print(len(output))
            fname_tag[fname] = output
    except Exception as err:
#         print(f'\t{err}', end='\r')
        fname_tag[fname] = np.nan
        err_log[e]=path
        e+=1
    
    # if idx ==500:
    #     break
        

In [5]:
set(path.parent for path in list(err_log.values()))

{PosixPath('../../data_raw/DearLep/MCTT')}

3810


In [6]:
import json

with open('meta/fname_tag.json', 'w') as fp:
    json.dump(fname_tag, fp)
    
with open('meta/fname_tag.json', 'r') as fp:
    fname_tag = json.load(fp)

print(len(list(fname_tag.items())))
list(fname_tag.items())[0]

3879


('Amphipyra fuscusa_male_Meifeng_20-VII-2012_SWu&WCChang_800px',
 ['Photo|吳士緯',
  'Stage|Adult',
  'Sex|Male',
  'Display|A',
  'Image_type|Specimen',
  'Deposition|TFRI',
  'Macroheterocera|Noctuidae',
  'Taxon|Macroheterocera|Noctuidae|Amphipyrinae|Amphipyra fuscusa',
  'Side|Dorsal'])

In [7]:

df_fname_tag = pd.DataFrame(fname_tag.items(), columns=['k','value'])
print(len(df_fname_tag))
df_fname_tag[df_fname_tag.k.str.startswith(".")]

3879


,k,value


In [38]:
df_tags = pd.DataFrame(columns=['Taxon','Sex', 'Name', 'Suborder', 'Family', 'Species'])
print(df_tags)
for idx, (k, v) in enumerate(fname_tag.items()):
    
    print(idx, k, end='\r')
    
    if type(v) == float:
        v = ['NaN|NaN']*5
    
    # if list所有的字串，以 '|'分割后的長度均為2:
    #   則以字串中含有科名的元素，前面加上'Taxon'
    if {len(str_.split('|'))  for str_ in v} == {2}: 
        v = ['Taxon|'+ str_ if 'dae' in str_ else str_ 
                for str_ in v ]
    
    v_dict = dict(
    zip(
        [str_.split('|')[0] for str_ in v],
        [str_.split('|', maxsplit=1)[1] for str_ in v])
    )
    
    if 'Sex' not in v_dict:
        v_dict['Sex'] = np.nan
    if 'Taxon' not in v_dict:
        v_dict['Taxon'] = np.nan 
    
    df_tags_ = pd.DataFrame.from_dict(v_dict,  orient='index').T
    df_tags_['Name'] =  k
    
    
    df_tags_ = df_tags_[['Taxon', 'Sex', 'Name']]
    
    if df_tags_.Taxon.notnull().values:
        taxon_list = df_tags_.Taxon.str.split('|').values[0]  
    else: 
        taxon_list = [np.nan]*3
    
    
    df_tags_['Suborder'] = taxon_list[0]
    df_tags_['Family'] = taxon_list[1]
    df_tags_['Species'] = taxon_list[-1] if not len(taxon_list)== 2 else np.nan
    
    
    # 改為建立新的字典、最後再轉成dataframe
    df_tags = pd.concat([df_tags, df_tags_]).reset_index(drop=True)

df_tags.to_csv('meta/tags_Taiwan_220105.csv')
df_tags

Empty DataFrame
Columns: [Taxon, Sex, Name, Suborder, Family, Species]
Index: []


,Taxon,Sex,Name,Suborder,Family,Species
0,Macroheterocera|Noctuidae|Amphipyrinae|Amphipy...,Male,Amphipyra fuscusa_male_Meifeng_20-VII-2012_SWu...,Macroheterocera,Noctuidae,Amphipyra fuscusa
1,Macroheterocera|Erebidae|Erebinae|Serrodes cam...,Female,Serrodes campana_female_Taian_26-XI-2016_SWu&W...,Macroheterocera,Erebidae,Serrodes campana
2,Macroheterocera|Erebidae|Arctiinae|Lithosiini|...,Male,Hesudra divisa_male_31mm_800,Macroheterocera,Erebidae,Hesudra divisa
3,Macroheterocera|Erebidae|Erebinae|Rema costima...,Female,Rema costimacula_female_MSE0500_19-20-XI-2015_...,Macroheterocera,Erebidae,Rema costimacula
4,Macroheterocera|Geometridae|Ennominae|Auaxa ce...,Female,Auaxa cesadaria_female_Biluxi_1-VII-2015_SSLu_...,Macroheterocera,Geometridae,Auaxa cesadaria
...,...,...,...,...,...,...
3874,Macroheterocera|Geometridae|Desmobathrinae|Ozo...,Male,Ozola defectata_male_Gangziliaoshan_14-VII-201...,Macroheterocera,Geometridae,Ozola defectata
3875,Macroheterocera|Noctuidae|Hadeninae|Polia mort...,Male,Polia mortua caeca_male_Hohuanshan_ESRI_001,Macroheterocera,Noctuidae,Polia mortua caeca
3876,Microlepidoptera|Sesiidae|Macroscelesia formosana,Male,NSMT_HT_Macroscelesia formosana Arita & Gorbun...,Microlepidoptera,Sesiidae,Macroscelesia formosana
3877,Macroheterocera|Erebidae|Boletobiinae|Diomea i...,Female,Diomea insulana_female_Tianwan_8-VI-2013_SWu_04,Macroheterocera,Erebidae,Diomea insulana


In [46]:
print(len(df_tags))
print(df_tags.query('Name.str.startswith(".").values').__len__())
print(df_tags.query('Taxon.isnull().values').__len__())
# df_tags.query('Name.str.startswith(".").values')



3879
0
75


In [42]:
df_tags = pd.read_csv('meta/tags_Taiwan_220105.csv', index_col=0)
df_tags


,Taxon,Sex,Name,Suborder,Family,Species
0,Macroheterocera|Noctuidae|Amphipyrinae|Amphipy...,Male,Amphipyra fuscusa_male_Meifeng_20-VII-2012_SWu...,Macroheterocera,Noctuidae,Amphipyra fuscusa
1,Macroheterocera|Erebidae|Erebinae|Serrodes cam...,Female,Serrodes campana_female_Taian_26-XI-2016_SWu&W...,Macroheterocera,Erebidae,Serrodes campana
2,Macroheterocera|Erebidae|Arctiinae|Lithosiini|...,Male,Hesudra divisa_male_31mm_800,Macroheterocera,Erebidae,Hesudra divisa
3,Macroheterocera|Erebidae|Erebinae|Rema costima...,Female,Rema costimacula_female_MSE0500_19-20-XI-2015_...,Macroheterocera,Erebidae,Rema costimacula
4,Macroheterocera|Geometridae|Ennominae|Auaxa ce...,Female,Auaxa cesadaria_female_Biluxi_1-VII-2015_SSLu_...,Macroheterocera,Geometridae,Auaxa cesadaria
...,...,...,...,...,...,...
3874,Macroheterocera|Geometridae|Desmobathrinae|Ozo...,Male,Ozola defectata_male_Gangziliaoshan_14-VII-201...,Macroheterocera,Geometridae,Ozola defectata
3875,Macroheterocera|Noctuidae|Hadeninae|Polia mort...,Male,Polia mortua caeca_male_Hohuanshan_ESRI_001,Macroheterocera,Noctuidae,Polia mortua caeca
3876,Microlepidoptera|Sesiidae|Macroscelesia formosana,Male,NSMT_HT_Macroscelesia formosana Arita & Gorbun...,Microlepidoptera,Sesiidae,Macroscelesia formosana
3877,Macroheterocera|Erebidae|Boletobiinae|Diomea i...,Female,Diomea insulana_female_Tianwan_8-VI-2013_SWu_04,Macroheterocera,Erebidae,Diomea insulana


- 過濾出缺乏類群欄位的名單給士緯

In [43]:
print(len(df_tags[df_tags.Taxon.isnull()]))

# df_tags[df_tags.Taxon.isnull()].Name.to_csv(dir_savemeta.joinpath('tags_null.csv'))
df_tags[df_tags.Taxon.isnull()]
print(df_tags.query('Species.isnull().values').__len__())
df_tags.query('Species.isnull().values')

75
101


,Taxon,Sex,Name,Suborder,Family,Species
12,NaN,NaN,Asota heliconia zebrina_male_Jianshi_9-VI-2018...,NaN,NaN,NaN
13,NaN,NaN,Syntypistis nigribasalis nigribasalis_male_A86...,NaN,NaN,NaN
14,NaN,NaN,Menophra mitsundoi_mana-removebg-preview,NaN,NaN,NaN
24,NaN,NaN,Bastilla praetermissa_male_A37-20160714-049-re...,NaN,NaN,NaN
30,NaN,NaN,Biston regalis comitata_male_V14-20130428-031-...,NaN,NaN,NaN
...,...,...,...,...,...,...
3048,Macroheterocera|Geometridae,Female,Protoboarmia amabilis_female_FBG_3-XII-2010_SW...,Macroheterocera,Geometridae,NaN
3090,NaN,NaN,Lomographa percnosticta_male_Guanwu_17-VI-2015...,NaN,NaN,NaN
3286,Macroheterocera|Erebidae,Female,Euplocia membliaria_NMNS 725-397-1,Macroheterocera,Erebidae,NaN
3572,Microlepidoptera|Crambidae,Male,Pseudargyria interruptella 黃紋銀草螟蛾_16mm_male_ES...,Microlepidoptera,Crambidae,NaN


- 依據士緯提供的'蝶蛾網學名tag確認_20211220.xlsx' 更新缺乏物種名的檔案

In [107]:
##  依據士緯提供的'蝶蛾網學名tag確認_20211220.xlsx' 更新缺乏物種名的檔案
sheet_Sp_null = pd.read_excel('meta/蝶蛾網學名tag確認_20211220.xlsx', sheet_name='2nd')

sheet_Sp_null.filename = sheet_Sp_null.filename.apply(lambda x: x.split(',', maxsplit=1)[-1])
sheet_Sp_null.filename = sheet_Sp_null.filename.apply(lambda x: x.strip('\"'))
sheet_Sp_null.query('species.isnull().values') 

,species,filename,Note
18,NaN,Mosopia lingmujaungi_male_HT_01_TFRI_800px,尚未發表、實驗未取樣，可略過，感謝


In [106]:
##  依據士緯提供的'MCTT_fname_null_SW20220107.xlsx 更新缺乏物種名的檔案
sheet_Sp_null_2 = pd.read_excel('meta/MCTT_fname_null_SW20220107.xlsx',engine='openpyxl',
                              names=['filename', 'species', 'Note']
                              )
sheet_Sp_null_2['filename'] = sheet_Sp_null_2.filename.apply(lambda x : x.split(',',maxsplit=1)[1])
sheet_Sp_null_2['filename'] = sheet_Sp_null_2.filename.apply(lambda x : x.strip(r'"'))
sheet_Sp_null_2 = sheet_Sp_null_2[['species', 'filename', 'Note']]
sheet_Sp_null_2.query('species.isnull().values') 
sheet_Sp_null_2.query("filename.str.contains('\"').values")


,species,filename,Note


In [108]:
sheet_Sp_null = pd.concat([sheet_Sp_null,sheet_Sp_null_2])
sheet_Sp_null.query('species.isnull().values') 

,species,filename,Note
18,NaN,Mosopia lingmujaungi_male_HT_01_TFRI_800px,尚未發表、實驗未取樣，可略過，感謝
88,NaN,Unknown-1_JRS00469_male,這隻是四川sJRS的


In [109]:
for idx, rows in sheet_Sp_null.iterrows() :
    sp, fname, _ = rows
    print(idx, sp, ':', fname, end='\r')
    index = df_tags[df_tags.Name == fname].index.values
    df_tags.loc[index,'Species'] = sp

df_tags[df_tags.Species.isnull()].Name.values

array(['Mosopia lingmujaungi_male_HT_01_TFRI_800px'], dtype=object)

In [110]:
df_tags.to_csv('meta/tags_Taiwan_220105.csv')
df_tags = pd.read_csv('meta/tags_Taiwan_220105.csv', index_col=0)
df_tags[df_tags.Species.isnull()]
# df_tags

,Taxon,Sex,Name,Suborder,Family,Species
1948,Macroheterocera|Erebidae,Male,Mosopia lingmujaungi_male_HT_01_TFRI_800px,Macroheterocera,Erebidae,NaN


In [96]:
print(df_tags[['Family', 'Species']].nunique())
df_tags.Species.unique()
df_tags.groupby(['Family', 'Species']).Taxon.count().sort_values()

Family       40
Species    1984
dtype: int64


Family         Species                      
Zygaenidae     Zeuxippa formosa                  1
Noctuidae      Hermonassa plantei                1
               Hermonassa inconstans             1
Geometridae    Atopophysa opulens                1
Noctuidae      Hermonassa hemicyclia             1
                                                ..
Geometridae    Abraxas suspecta                  9
Lasiocampidae  Kunugia undans metanastroides    13
Geometridae    Abraxas formosilluminata         14
               Abraxas tenellula                14
               Abraxas adilluminata             15
Name: Taxon, Length: 1985, dtype: int64

In [97]:
# df_tags.to_csv(dir_savemeta.joinpath('MC_DearLep_specimen-image_20211203_tags.csv'))
df_tags.describe()

# 僅有一筆資料缺乏物種名 : Mosopia lingmujaungi_male_HT_01_TFRI_800px

,Taxon,Sex,Name,Suborder,Family,Species
count,3804,3790,3810,3804,3804,3805
unique,1994,3,3810,3,40,1984
top,Macroheterocera|Geometridae|Ennominae|Abraxas ...,Male,Barsine karekonis_female_Cien_19-VII-2012_TFRI...,Macroheterocera,Geometridae,Abraxas adilluminata
freq,15,2326,1,3526,1024,15


#### load df_MC_SpCode_imgpath

In [252]:
df_MC_SpCode_imgpath = pd.read_csv('meta/df_MC_SpCode_imgpath.csv', index_col=0)
df_MC_SpCode_imgpath

,Family,Species,Code,namecode,species,path_id,path,path_count
0,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3638.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
1,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3675.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...,NaN
2,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3692.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/00 Spec/H...,NaN
3,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3757.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
4,Noctuidae,Hermonassa legraini,346335,346335,Hermonassa legraini,3781.0,./images/Noctuoidea 夜蛾總科/NOC/NOC_NOC/120814 南投...,NaN
...,...,...,...,...,...,...,...,...
67,Pyralidae,Sybrida inordinata,347546,347546,Sybrida inordinata,NaN,NaN,0.0
68,Erebidae,Sypnoides simplex,346778,346778,Sypnoides simplex,NaN,NaN,0.0
69,Pyralidae,Teliphasa amica,347552,347552,Teliphasa amica,NaN,NaN,0.0
70,Crambidae,Tenerobotys subfumalis,344394,344394,Tenerobotys subfumalis,NaN,NaN,0.0


- 使用科名、物種名來比對

In [253]:
df_MC_SpCode_count = pd.read_csv('meta/df_MC_path_count.csv', index_col=0)
df_MC_SpCode_count

,Family,Species,Code,path_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0
1,Geometridae,"""Collix"" sp.",NaN,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0
...,...,...,...,...
966,Noctuidae,Xylostola indistincta,346837,1
967,Geometridae,Yashmakia suffusa,345147,3
968,Erebidae,Zanclognatha nigrisigna,346843,4
969,Erebidae,Zanclognatha sp.,NaN,0


In [256]:
df_MC_tags = df_MC_SpCode_count.merge(df_tags[['Family','Species', 'Name']], 
                                            left_on=['Family','Species'], right_on=['Family','Species'], how='left'  ) 
df_MC_tags

,Family,Species,Code,path_count,Name
0,Erebidae,"""Adrapsa"" sp.",NaN,0,NaN
1,Geometridae,"""Collix"" sp.",NaN,0,NaN
2,Erebidae,"""Eilema"" sp. 1",NaN,0,NaN
3,Erebidae,"""Eilema"" sp. 2",NaN,0,NaN
4,Erebidae,"""Eilema"" sp. 3",NaN,0,NaN
...,...,...,...,...,...
1572,Geometridae,Yashmakia suffusa,345147,3,GEO_ENN_S_Yashmakia suffusa_male_Wulai_15-III-...
1573,Erebidae,Zanclognatha nigrisigna,346843,4,Zanclognatha nigrisigna_female_TFRI_800px
1574,Erebidae,Zanclognatha nigrisigna,346843,4,Zanclognatha nigrisigna_male_TFRI_800px
1575,Erebidae,Zanclognatha sp.,NaN,0,NaN


In [259]:
df_MC_tags_count = df_MC_tags.groupby(['Family','Species']).count().Name.rename('tags_count')
df_MC_tags_count

df_MC_tags_count = df_MC_SpCode_count.merge(df_MC_tags_count,
                                            left_on=['Family','Species'], right_index=True, how='left' )
df_MC_tags_count.to_csv('meta/MC_tags_count.csv')
df_MC_tags_count

,Family,Species,Code,path_count,tags_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0,0
1,Geometridae,"""Collix"" sp.",NaN,0,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0,0
...,...,...,...,...,...
966,Noctuidae,Xylostola indistincta,346837,1,1
967,Geometridae,Yashmakia suffusa,345147,3,1
968,Erebidae,Zanclognatha nigrisigna,346843,4,2
969,Erebidae,Zanclognatha sp.,NaN,0,0


In [262]:
df_MC_tags_count.query('tags_count==0')

,Family,Species,Code,path_count,tags_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0,0
1,Geometridae,"""Collix"" sp.",NaN,0,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0,0
...,...,...,...,...,...
950,Crambidae,Tyspanodes striata,344406,0,0
957,Geometridae,Xanthorhoe mediofascia,345359,1,0
958,Geometridae,Xanthorhoe saturata,345360,2,0
969,Erebidae,Zanclognatha sp.,NaN,0,0


In [263]:
df_MC_tags_count.query('tags_count>0')

,Family,Species,Code,path_count,tags_count
10,Geometridae,Abaciscus costimacula,344582,6,3
12,Erebidae,Aberrasine lichenshihi,NaN,0,2
13,Geometridae,Abraxas adilluminata,344585,15,14
14,Geometridae,Abraxas consputa,344587,5,4
16,Geometridae,Abraxas persimplex,344593,6,3
...,...,...,...,...,...
964,Noctuidae,Xestia supravidua,428181,3,2
965,Noctuidae,Xestia tamsi,346826,4,1
966,Noctuidae,Xylostola indistincta,346837,1,1
967,Geometridae,Yashmakia suffusa,345147,3,1


#### 小結:
- MC_DearLep_specimen-image_20211203有物種名的資料共3629筆，合計有1983個物種，其中只有621種，有在MC的物種清單中

## 三、特生蝶類標本資料比對


##### 資料來源:
- gbif: [Data-set of Moth Specimen from TESRI](https://www.gbif.org/dataset/f3f25fcf-2930-4cf1-a495-6b31d7fa0252)
    - 'verbatim.txt'

##### 工作流程:
1. 根據上一節'MC_tags_count.csv'比對結果，篩選出目前缺少的物種名單
    - 也整理一份根據所有物種比對的清單
2. 優先過濾出特生也沒有的物種名錄提供給士緯
    - 有路徑不代表有圖像
    - 圖像下載後還要過濾、不見得能用
    - 下載後比對仍然缺乏的

- 注意事項:
    - (從特生資料補充下載目前已經有的影像?)
    - 資料篩選時避免與forYY資料重疊做白工!

     



#### 資料前處理與合併

- 讀取特生蛾類標本清單 'verbatim.txt'

In [233]:
df_verbatim = pd.read_csv('data/verbatim.txt', sep='\t')
print(f'Number of data : {len(df_verbatim):,d}')
print('Number of columns : ', len(df_verbatim.columns.values))

# drop columns which all value is NaN 
df_verbatim.dropna(axis=1, how='all', inplace=True)
print(f'Number of columns : {len(df_verbatim.columns.values)} (after drop na by columns) ', )

# drop 'scientificName' column which value is NaN 
df_verbatim.dropna(subset=['scientificName'], inplace=True)
print(f'Number of data : {len(df_verbatim):,d} (after drop na by "scientificName" column)')

# drop 'associatedMedia column which value is NaN 
df_verbatim.dropna(subset=['associatedMedia'], inplace=True)
print(f'Number of data : {len(df_verbatim):,d} (after drop na by "associatedMedia" column)')

print('\ncolumns : ', df_verbatim.columns.values)
df_verbatim


Number of data : 109,653
Number of columns :  219
Number of columns : 36 (after drop na by columns) 
Number of data : 81,479 (after drop na by "scientificName" column)
Number of data : 81,229 (after drop na by "associatedMedia" column)

columns :  ['gbifID' 'identifier' 'collectionID' 'institutionCode' 'basisOfRecord'
 'occurrenceID' 'individualCount' 'sex' 'lifeStage' 'associatedMedia'
 'associatedReferences' 'eventID' 'parentEventID' 'eventDate'
 'samplingProtocol' 'sampleSizeValue' 'sampleSizeUnit' 'country'
 'countryCode' 'county' 'municipality' 'locality' 'verbatimElevation'
 'decimalLatitude' 'decimalLongitude' 'geodeticDatum' 'identifiedBy'
 'scientificName' 'kingdom' 'phylum' 'class' 'order' 'family' 'genus'
 'taxonRank' 'vernacularName']


,gbifID,identifier,collectionID,institutionCode,basisOfRecord,occurrenceID,individualCount,sex,lifeStage,associatedMedia,...,identifiedBy,scientificName,kingdom,phylum,class,order,family,genus,taxonRank,vernacularName
0,1934768067,A36-20130606-040,A36-20130606-040,tesri,PreservedSpecimen,A36-20130606-040,1,unknow,adult,http://farm8.staticflickr.com/7429/9554379312_...,...,施禮正,Chrysaeglia magnifica,Animalia,Arthropoda,Insecta,Lepidoptera,Erebidae,Chrysaeglia,species,閃光苔蛾
1,1934768377,A36-20130606-041,A36-20130606-041,tesri,PreservedSpecimen,A36-20130606-041,1,unknow,adult,http://farm6.staticflickr.com/5479/9551585327_...,...,施禮正,Spilarctia nydia werneri,Animalia,Arthropoda,Insecta,Lepidoptera,Erebidae,Spilarctia,subspecies,黑鬚污燈蛾
3,1934767936,A36-20130606-043,A36-20130606-043,tesri,PreservedSpecimen,A36-20130606-043,1,unknow,adult,http://farm4.staticflickr.com/3671/9554375768_...,...,施禮正,Barsine sauteri,Animalia,Arthropoda,Insecta,Lepidoptera,Erebidae,Barsine,species,東方葩苔蛾
4,1934767969,A36-20130606-048,A36-20130606-048,tesri,PreservedSpecimen,A36-20130606-048,1,unknow,adult,http://farm8.staticflickr.com/7446/9554369076_...,...,施禮正,Narosoideus vulpina,Animalia,Arthropoda,Insecta,Lepidoptera,Limacodidae,Narosoideus,species,狡娜刺蛾
5,1934768183,A36-20130606-049,A36-20130606-049,tesri,PreservedSpecimen,A36-20130606-049,1,unknow,adult,http://farm4.staticflickr.com/3757/9551574537_...,...,施禮正,Xyleutes strix,Animalia,Arthropoda,Insecta,Lepidoptera,Cossidae,Xyleutes,species,梟斑蠹蛾
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109648,2550500679,V38-20191102-067,V38-20191102-067,tesri,PreservedSpecimen,V38-20191102-067,1,femal,adult,https:\/\/live.staticflickr.com\/65535\/493925...,...,林旭宏,Alcis taiwanovariegata,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Alcis,species,臺灣暗斑霜尺蛾
109649,1934783697,A52-20160704-020,A52-20160704-020,tesri,PreservedSpecimen,A52-20160704-020,1,unknow,adult,https://farm8.staticflickr.com/7741/2902005066...,...,林旭宏,Chrysorabdia vilemani,Animalia,Arthropoda,Insecta,Lepidoptera,Erebidae,Chrysorabdia,species,雙帶苔蛾
109650,2550500777,V38-20191102-066,V38-20191102-066,tesri,PreservedSpecimen,V38-20191102-066,1,unknow,adult,https:\/\/live.staticflickr.com\/65535\/493931...,...,林旭宏,Fascellina chromataria,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Fascellina,species,褐缺口尺蛾
109651,2550500710,V38-20191102-065,V38-20191102-065,tesri,PreservedSpecimen,V38-20191102-065,1,femal,adult,https:\/\/live.staticflickr.com\/65535\/493931...,...,林旭宏,Orthocabera sericea sericea,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Orthocabera,subspecies,山茶斜帶尺蛾


In [264]:
columns_sel = ['gbifID', 'collectionID', 'family', 'genus', 'scientificName', 'sex', 'associatedMedia']
df_verbatim_ = df_verbatim[columns_sel]
df_verbatim_['associatedMedia'] = (df_verbatim_.associatedMedia
                                   .apply(lambda x : str(x).replace('\\', ''))
                                   .apply(lambda str_ : str_.replace('_q.jpg', '.jpg'))
                                   )
print(len(df_verbatim_))
df_verbatim_.head(3)

81229


C:\Users\yunghui\anaconda3\envs\pytorch_as\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,gbifID,collectionID,family,genus,scientificName,sex,associatedMedia
0,1934768067,A36-20130606-040,Erebidae,Chrysaeglia,Chrysaeglia magnifica,unknow,http://farm8.staticflickr.com/7429/9554379312_...
1,1934768377,A36-20130606-041,Erebidae,Spilarctia,Spilarctia nydia werneri,unknow,http://farm6.staticflickr.com/5479/9551585327_...
3,1934767936,A36-20130606-043,Erebidae,Barsine,Barsine sauteri,unknow,http://farm4.staticflickr.com/3671/9554375768_...


- 與df_MC 資料合併

In [282]:
df_MC_tags_count = pd.read_csv('meta/MC_tags_count.csv', 
                               index_col=0)
df_MC_tags_count


,Family,Species,Code,path_count,tags_count
0,Erebidae,"""Adrapsa"" sp.",NaN,0,0
1,Geometridae,"""Collix"" sp.",NaN,0,0
2,Erebidae,"""Eilema"" sp. 1",NaN,0,0
3,Erebidae,"""Eilema"" sp. 2",NaN,0,0
4,Erebidae,"""Eilema"" sp. 3",NaN,0,0
...,...,...,...,...,...
966,Noctuidae,Xylostola indistincta,346837,1,1
967,Geometridae,Yashmakia suffusa,345147,3,1
968,Erebidae,Zanclognatha nigrisigna,346843,4,2
969,Erebidae,Zanclognatha sp.,NaN,0,0


- 產出關連到特生影像網址的頁面清單

In [322]:
df_MC_tersi =  (df_MC_tags_count[['Family', 'Species','tags_count']]
                .merge(df_verbatim_[['collectionID','family','scientificName','associatedMedia']],
                       left_on=['Family','Species'], 
                       right_on=['family','scientificName'], 
                       how='left')
                )
df_MC_tersi.dropna(subset=['collectionID'], inplace=True)
assert df_MC_tersi.query('associatedMedia.isnull()').__len__() == 0
df_MC_tersi.reset_index(drop=True, inplace=True)

df_MC_tersi.to_csv('meta/MC_tersi_imgpath')
df_MC_tersi = pd.read_csv('meta/MC_tersi_imgpath', index_col=0)
df_MC_tersi


,Family,Species,tags_count,collectionID,family,scientificName,associatedMedia
0,Geometridae,Abaciscus costimacula,3,A58-20180716-150,Geometridae,Abaciscus costimacula,https://farm2.staticflickr.com/1884/2925080974...
1,Geometridae,Abaciscus costimacula,3,A58-20180716-149,Geometridae,Abaciscus costimacula,https://farm2.staticflickr.com/1860/2925081045...
2,Geometridae,Abaciscus costimacula,3,V43-20201219-061,Geometridae,Abaciscus costimacula,https://live.staticflickr.com/65535/5087961433...
3,Geometridae,Abaciscus costimacula,3,V41-20200905-085,Geometridae,Abaciscus costimacula,https://live.staticflickr.com/65535/5070012011...
4,Geometridae,Abaciscus costimacula,3,A55-20210511-035,Geometridae,Abaciscus costimacula,https://live.staticflickr.com/65535/5153280201...
...,...,...,...,...,...,...,...
42528,Cossidae,Zeuzera multistrigata,0,V04-20140721-238,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7341/1619214443...
42529,Cossidae,Zeuzera multistrigata,0,V04-20140816-118,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7583/1572573768...
42530,Cossidae,Zeuzera multistrigata,0,A58-20170830-070,Cossidae,Zeuzera multistrigata,https://farm5.staticflickr.com/4497/3752616800...
42531,Cossidae,Zeuzera multistrigata,0,A09-20090712-041,Cossidae,Zeuzera multistrigata,http://farm3.staticflickr.com/2527/3881182708_...


In [2]:
# 計算各物主對應的特生影像數量
# count = (df_MC_tersi.groupby(['Family', 'Species'])
#          .associatedMedia.count()
#          .rename('tersi_count'))
# count 
# df_MC_tersi_count = (df_MC_tags_count[['Family','Species','tags_count']]
#                     .merge(count,
#                         left_on=['Family','Species'], 
#                         right_index=True, 
#                         how='left'
#                             )
# )
# df_MC_tersi_count.to_csv('meta/MC_tersi_count.csv')
df_MC_tersi_count = pd.read_csv('meta/MC_tersi_count.csv', index_col=0)
df_MC_tersi_count


,Family,Species,tags_count,tersi_count
0,Erebidae,"""Adrapsa"" sp.",0,0
1,Geometridae,"""Collix"" sp.",0,0
2,Erebidae,"""Eilema"" sp. 1",0,0
3,Erebidae,"""Eilema"" sp. 2",0,0
4,Erebidae,"""Eilema"" sp. 3",0,0
...,...,...,...,...
966,Noctuidae,Xylostola indistincta,1,84
967,Geometridae,Yashmakia suffusa,1,46
968,Erebidae,Zanclognatha nigrisigna,2,18
969,Erebidae,Zanclognatha sp.,0,0


#### 找出"MC_DearLep_specimen-image_20211203"資料夾內缺少、特生資料庫亦缺少的物種清單

In [329]:
# 確認 "MC_DearLep_specimen-image_20211203"資料夾內無資料、且特生資料有影像的物種清單、物種名不含"sp." 
assert  df_MC_tersi_count.query('tags_count==0 and tersi_count>0 and Species.str.contains("sp\.")').size == 0
print('"MC_DearLep_specimen-image_20211203"資料夾內缺少的物種數 : ',
    df_MC_tersi_count.query('tags_count==0 ').__len__()
    )

print('"MC_DearLep_specimen-image_20211203"資料夾內缺少， ')
print('\t- 特生資料庫有的物種種數 : ',
      df_MC_tersi_count.query('tags_count==0 and tersi_count>0').__len__()
      )
print('\t- 特生資料庫亦缺少的物種種數 : ',
      df_MC_tersi_count.query('tags_count==0 and tersi_count.isnull()').__len__()
      )


"MC_DearLep_specimen-image_20211203"資料夾內缺少的物種數 :  350
"MC_DearLep_specimen-image_20211203"資料夾內缺少， 
	- 特生資料庫有的物種種數 :  188
	- 特生資料庫亦缺少的物種種數 :  162


- "MC_DearLep_specimen-image_20211203"資料夾內缺少、特生資料庫亦缺少的物種清單

In [3]:
#　"MC_DearLep_specimen-image_20211203"資料夾內缺少、特生資料庫亦缺少的物種清單
# df_MC_tersi_null = df_MC_tersi_count.query('tags_count==0 and tersi_count.isnull()')
# df_MC_tersi_null = df_MC_tersi_null.merge(df_MC_SpCode,
#                         left_on=['Family','Species'], 
#                         right_on=['Family','Species'], 
#                         how='left')[['Family','Species','Code']]
# df_MC_tersi_null.to_csv('meta/MC_tersi_null')
df_MC_tersi_null = pd.read_csv('meta/MC_tersi_null', index_col=0)

df_MC_tersi_null

,Family,Species,Code
0,Erebidae,"""Adrapsa"" sp.",NaN
1,Geometridae,"""Collix"" sp.",NaN
2,Erebidae,"""Eilema"" sp. 1",NaN
3,Erebidae,"""Eilema"" sp. 2",NaN
4,Erebidae,"""Eilema"" sp. 3",NaN
...,...,...,...
157,Pyralidae,Teliphasa sp.,NaN
158,Crambidae,Teliphasa sp.,NaN
159,Crambidae,Tenerobotys subfumalis,344394
160,Tortricidae,Terthreutis sp.,NaN


- "MC_DearLep_specimen-image_20211203"資料夾內缺少、特生資料庫有的物種清單

In [27]:
# df_MC_tersi_tagnull_imgpath = df_MC_tersi.query('tags_count==0 and associatedMedia.notnull()').reset_index(drop=True)
# df_MC_tersi_tagnull_imgpath.to_csv('meta/MC_tersi_tagnull_imgpath.csv')
df_MC_tersi_tagnull_imgpath = pd.read_csv('meta/MC_tersi_tagnull_imgpath.csv', index_col=0)
df_MC_tersi_tagnull_imgpath

,Family,Species,tags_count,collectionID,family,scientificName,associatedMedia
0,Geometridae,Abaciscus tristis tristis,0,V42-20201010-137,Geometridae,Abaciscus tristis tristis,https://live.staticflickr.com/65535/5096177997...
1,Geometridae,Abaciscus tristis tristis,0,V42-20201010-156,Geometridae,Abaciscus tristis tristis,https://live.staticflickr.com/65535/5096097241...
2,Geometridae,Abaciscus tristis tristis,0,V40-20200117-035,Geometridae,Abaciscus tristis tristis,https://live.staticflickr.com/65535/4965360088...
3,Geometridae,Abaciscus tristis tristis,0,A58-20180716-157,Geometridae,Abaciscus tristis tristis,https://farm2.staticflickr.com/1859/4237997025...
4,Geometridae,Abaciscus tristis tristis,0,V42-20200222-052,Geometridae,Abaciscus tristis tristis,https://live.staticflickr.com/65535/4976786250...
...,...,...,...,...,...,...,...
10265,Cossidae,Zeuzera multistrigata,0,V04-20140721-238,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7341/1619214443...
10266,Cossidae,Zeuzera multistrigata,0,V04-20140816-118,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7583/1572573768...
10267,Cossidae,Zeuzera multistrigata,0,A58-20170830-070,Cossidae,Zeuzera multistrigata,https://farm5.staticflickr.com/4497/3752616800...
10268,Cossidae,Zeuzera multistrigata,0,A09-20090712-041,Cossidae,Zeuzera multistrigata,http://farm3.staticflickr.com/2527/3881182708_...


- 比對實際下載到的影像

In [30]:
# 檢視沒有下載到的影像id
logging = pd.read_csv('meta/logging_MC_tersi_tagnull_imgpath_download_fail.txt',
                      header=None,
                      index_col=0,
                      names=['id','path'])
logging = logging.apply(lambda df : df.str.strip())
logging = (logging.merge(df_MC_tersi_tagnull_imgpath,
              left_on=['id'],
              right_on=['collectionID'],
              how='left')
           [['Family', 'Species', 'id', 'path']])
logging

,Family,Species,id,path
0,Notodontidae,Allodontoides tenebrosa,A43-20170814-150,https://farm5.staticflickr.com/4379/3706553622...
1,Drepanidae,Microblepsis violacea,A01-20091021-050,http://farm5.staticflickr.com/4041/4701944444_...
2,Limacodidae,Narosa nigrisigna,A43-20170814-171,https://farm5.staticflickr.com/4337/3706557463...
3,Nolidae,Negeta signata,A43-20170814-185,https://farm5.staticflickr.com/4362/3683963123...
4,Drepanidae,Oreta loochooana,A01-20091222-088,http://farm5.staticflickr.com/4122/4829532351_...
5,Crambidae,Palpita hypohomalia,A01-20090924-104,http://farm4.staticflickr.com/3342/4643784475_...
6,Crambidae,Paracymoriza taiwanalis,A09-20090824-045,http://farm3.staticflickr.com/2726/4149157555_...
7,Crambidae,Polythlipta divaricata,A06-20090917-110,http://farm5.staticflickr.com/4038/4252147827_...
8,Crambidae,Polythlipta divaricata,A01-20091222-103,http://farm5.staticflickr.com/4141/4829587903_...
9,Thyrididae,Pyrinioides sinuosus,A43-20170814-172,https://farm5.staticflickr.com/4378/3723778779...


In [ ]:
#　從df_MC_tersi_tagnull_imgpath清單比對、刪除未下載到的影像

In [55]:
index_deleted = logging.index.values
index_deleted
df_MC_tersi_tagnull_imgpath_ = df_MC_tersi_tagnull_imgpath.drop(index=index_deleted)
df_MC_tersi_tagnull_imgpath_


,Family,Species,tags_count,collectionID,family,scientificName,associatedMedia
15,Geometridae,Abaciscus tristis tristis,0,A41-20180226-139,Geometridae,Abaciscus tristis tristis,https://farm1.staticflickr.com/869/40662826644...
16,Geometridae,Abaciscus tristis tristis,0,V24-20190413-123,Geometridae,Abaciscus tristis tristis,https://farm66.staticflickr.com/65535/46949766...
17,Geometridae,Abaciscus tristis tristis,0,A58-20181007-066,Geometridae,Abaciscus tristis tristis,https://farm2.staticflickr.com/1938/3198197230...
18,Geometridae,Abaciscus tristis tristis,0,V03-20130510-174,Geometridae,Abaciscus tristis tristis,http://farm6.staticflickr.com/5463/9250492467_...
19,Geometridae,Abaciscus tristis tristis,0,A32-20120725-019,Geometridae,Abaciscus tristis tristis,http://farm9.staticflickr.com/8182/8065868115_...
...,...,...,...,...,...,...,...
10265,Cossidae,Zeuzera multistrigata,0,V04-20140721-238,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7341/1619214443...
10266,Cossidae,Zeuzera multistrigata,0,V04-20140816-118,Cossidae,Zeuzera multistrigata,https://farm8.staticflickr.com/7583/1572573768...
10267,Cossidae,Zeuzera multistrigata,0,A58-20170830-070,Cossidae,Zeuzera multistrigata,https://farm5.staticflickr.com/4497/3752616800...
10268,Cossidae,Zeuzera multistrigata,0,A09-20090712-041,Cossidae,Zeuzera multistrigata,http://farm3.staticflickr.com/2527/3881182708_...


In [41]:
logging_count = logging.groupby(['Family','Species']).count().id.rename('tersi_count_deleted')
logging_count

Family        Species                
Crambidae     Palpita hypohomalia        1
              Paracymoriza taiwanalis    1
              Polythlipta divaricata     2
              Tyspanodes striata         2
Drepanidae    Microblepsis violacea      1
              Oreta loochooana           1
Erebidae      Euproctis croceola         1
              Rusicada fulvida           1
Limacodidae   Iraga rugosa               1
              Narosa nigrisigna          1
Nolidae       Negeta signata             1
Notodontidae  Allodontoides tenebrosa    1
Thyrididae    Pyrinioides sinuosus       1
Name: tersi_count_deleted, dtype: int64

In [61]:
df_MC_tersi_count_ = df_MC_tersi_count.merge(logging_count,
                                             left_on=['Family','Species'],
                                             right_index=True,
                                             how='left')
df_MC_tersi_count_.tersi_count_deleted.fillna(0,inplace=True)
df_MC_tersi_count_.eval('tersi_count - tersi_count_deleted')
df_MC_tersi_count_.iloc[index_deleted]

,Family,Species,tags_count,tersi_count,tersi_count_deleted
0,Erebidae,"""Adrapsa"" sp.",0,0,0.0
1,Geometridae,"""Collix"" sp.",0,0,0.0
2,Erebidae,"""Eilema"" sp. 1",0,0,0.0
3,Erebidae,"""Eilema"" sp. 2",0,0,0.0
4,Erebidae,"""Eilema"" sp. 3",0,0,0.0
5,Noctuidae,"""Koyaga"" sp._blue",0,0,0.0
6,Geometridae,"""Lophophleps"" sp.",0,0,0.0
7,Erebidae,"""Plecoptera"" sp.",0,0,0.0
8,Noctuidae,"""Rivula"" sp.",0,0,0.0
9,Geometridae,"""Syncosmia"" sp.",0,0,0.0


In [59]:
df_MC_tersi_count.iloc[index_deleted]

,Family,Species,tags_count,tersi_count
0,Erebidae,"""Adrapsa"" sp.",0,0
1,Geometridae,"""Collix"" sp.",0,0
2,Erebidae,"""Eilema"" sp. 1",0,0
3,Erebidae,"""Eilema"" sp. 2",0,0
4,Erebidae,"""Eilema"" sp. 3",0,0
5,Noctuidae,"""Koyaga"" sp._blue",0,0
6,Geometridae,"""Lophophleps"" sp.",0,0
7,Erebidae,"""Plecoptera"" sp.",0,0
8,Noctuidae,"""Rivula"" sp.",0,0
9,Geometridae,"""Syncosmia"" sp.",0,0
